# __LightGBM__
É um algoritmo baseado em boosting. Sua ideia é é ser um algoritmo eficiente e escalável, principalmente quando lidando com conjuntos grandes de dados.

## __Conceito__
O LightGBM (_Light Gradient Boosting Machine_) tem como princípio melhorar o desempenho e eficiência do Gradient Boosting tradicional que aplicamos anteriormente. Para relembrar, o _gradient boosting_ é uma técnica que combina diversos modelos fracos (que geralmente são árvores de decisão) para formar um modelo forte. Cada modelo seguinte tem como objetivo corrigir os erros dos modelos anteriores, ajustando-se aos resíduos.

A intuição dele é de otimizar esse processo de correção. Para esse objetivo são aplicadas algumas técnicas, como:
- __Crescimento de árvore baseado em folhas (leaf-wise growth):__ Ao invés de expandir as árvores de forma nivelada (depth-wise), o LightGBM cresce as árvores de forma assimétrica, expandindo a folha que resulta na maior redução de perda.
- __Amostragem baseada em gradiente (GOSS):__ Foca nas instâncias com maiores gradientes, considerando que elas contribuem mais para o cálculo da perda.
- __Técnicas de binning:__ Reduz a complexidade computacional ao discretizar valores contínuos em bins.

## __Funcionamento__
- 1) __Inicialização:__ Começa com uma previsão inicial, geralmente sendo a média dos valores.
- 2) __Cálculo dos Resíduos:__ A cada nova iteração, calcula os resíduoes (que é a diferença entre o valor real e previsto).
- 3) __Treinamento da Árvore:__ Treina uma nova árvore para prever os resíduos calculados.
- 4) __Atualização do Modelo:__ As previsões então são atualizadas adicionando uma fração (learning rate) das previsões da nova árvore ao modelo existente.
- 5) __Iterações:__ Os passos 2 e 4 são repetidos até atingir o número de iterações que determinamos.

## __Processamento e pré-processamento__
- __Codificação de variáveis categóricas:__ O LightGBM é capaz de lidar com variáveis categóricas, de forma a evitar a necessidade de OHE ou outra técnica nesse sentido.
- __Remoção de outliers:__ Não é totalmente necessário, mas pode ajudar a melhorar o desempenho do modelo.
- __Balanceamento de classes:__ Para problemas de classificação com dados desbalanceados, pode ser uma boa aplicar técnicas de reamostragem, ou aplicar ajuste de peso das classes.

## __Vantagens__
- 1) __Velocidade:__ Devido a otimizações como o binning e crescimento baseado em folhas, o treinamento ocorre de uma forma mais veloz.
- 2) __Escalabilidade:__ É capaz de lidar bem com grandes volumes de dados.
- 3) __Variáveis Categóricas:__ É capaz de trabalhar com variáveis categóricas.
- 4) __Precisão:__ É um algoritmo que apresentou ótimas performances em competições.

## __Detalhes adicionais__
- __Crescimento Baseado em Folhas:__ Algoritmos como XGBoost realizam o crescimento das árvores de forma nivelada (deep-wise). Isso quer dizer que as folhas em um nível são divididas antes de passar para o próximo nível, o que gera árvores simétricas. Já no __LightGBM__, a expansão da árvore ocorre dividindo sempre a folha que resulta na maior redução da função de perda. Isso acontece independente do nível da folha, o que gera um crescimento assimétrico. Isso permite que seja focado nas folhas com a maior redução de perda, melhorando o desempenho do modelo. Também permite que o modelo seja capaz de perceber algumas interações mais complexas entre as variáveis. Devido a aplicação desse método, as árvores podem acabar sendo mais profundas em uma área ou outra, fazendo com que seja realizado um ajuste mais fino onde for necessário.

In [1]:
# Importação do google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Importações
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import warnings
warnings.filterwarnings('ignore')
plt.style.use('ggplot')

In [4]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, RandomizedSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [5]:
alvo = pd.read_pickle('/content/drive/MyDrive/Udemy/ML com Python/1 - Aprendizado Supervisionado: Classificacao/heart.pkl')

# Variáveis previsoras onde as variáveis categóricas foram transformadas em numéricas manualmente, sem escalonamento
previsores = pd.read_pickle('/content/drive/MyDrive/Udemy/ML com Python/1 - Aprendizado Supervisionado: Classificacao/heart2.pkl')

# previsores_esc = pd.read_pickle('/content/drive/MyDrive/Udemy/ML com Python/1 - Aprendizado Supervisionado: Classificacao/heart3.pkl')

# Variáveis previsoras onde as variáveis categóricas foram transformadas em numéricas pelo LabelEncoder.
previsores2 = pd.read_pickle('/content/drive/MyDrive/Udemy/ML com Python/1 - Aprendizado Supervisionado: Classificacao/heart4.pkl')

# Variáveis previsoras onde as variáveis categóricas foram transformadas em numéricas pelo LabelEncoder e OneHotEncoder, sem escalonamento.
previsores3 = pd.read_pickle('/content/drive/MyDrive/Udemy/ML com Python/1 - Aprendizado Supervisionado: Classificacao/heart5.pkl')

# Variáveis previsoras onde as variáveis categóricas foram transformadas pelo LabelEncoder e OHE, com escalonamento.
previsores3_esc = pd.read_pickle('/content/drive/MyDrive/Udemy/ML com Python/1 - Aprendizado Supervisionado: Classificacao/heart6.pkl')

In [6]:
X_tr, X_ts, y_tr, y_ts = train_test_split(previsores, alvo, test_size=.3, random_state=42)

In [7]:
# Criação do Dataset de Treino
dataset = lgb.Dataset(X_tr, label=y_tr)

In [9]:
X_tr.shape

(641, 11)

In [10]:
X_ts.shape

(276, 11)

In [11]:
y_tr.shape

(641,)

In [12]:
y_ts.shape

(276,)

In [13]:
# Parâmetros que iremos testas
params = {
    'num_leaves': 150,
    'objective': 'binary',
    'max_depth': 7,
    'learning_rate': 0.05,
    'max_bin': 200
}

In [14]:
lgbm = lgb.train(params, dataset, num_boost_round=150)

[LightGBM] [Info] Number of positive: 338, number of negative: 303
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000482 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 408
[LightGBM] [Info] Number of data points in the train set: 641, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.527301 -> initscore=0.109313
[LightGBM] [Info] Start training from score 0.109313
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

In [15]:
from inspect import EndOfBlock
# Verificação do tempo de execução
from datetime import datetime
start = datetime.now()
lgbm = lgb.train(params, dataset, num_boost_round=150)
end = datetime.now()

print(end-start)

[LightGBM] [Info] Number of positive: 338, number of negative: 303
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000121 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 408
[LightGBM] [Info] Number of data points in the train set: 641, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.527301 -> initscore=0.109313
[LightGBM] [Info] Start training from score 0.109313
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

In [16]:
# Realizar previsões
y_pred = lgbm.predict(X_ts)

In [17]:
y_pred.shape

(276,)

In [18]:
# Precisamos definir agora o threshold da classificação
for i in range(0, len(y_pred)):
  if y_pred[i] >= 0.5:
    y_pred[i] = 1
  else:
    y_pred[i] = 0

In [19]:
y_pred

array([0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0.,
       1., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1.,
       1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 0., 1.,
       0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1.,
       1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1.,
       0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 0., 0.,
       1., 0., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0.,
       0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 1., 0.,
       0., 0., 0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1.,
       0., 1., 0., 1., 0., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1.,
       0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1.,
       1., 1., 0., 1., 0.

In [21]:
# Verificação da acurácia
print(f'Acurácia: {accuracy_score(y_ts, y_pred)*100:.3f}')

Acurácia: 87.681


In [22]:
# Exibir a matriz de confusão
confusion_matrix(y_ts, y_pred)

array([[ 93,  14],
       [ 20, 149]])

In [25]:
# Exibir o report de classificação
print(classification_report(y_ts, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.87      0.85       107
           1       0.91      0.88      0.90       169

    accuracy                           0.88       276
   macro avg       0.87      0.88      0.87       276
weighted avg       0.88      0.88      0.88       276



## Teste de Diferentes Parâmetros e Validação Cruzada

In [26]:
# Parâmetros que vamos testar
params = {
    'num_leaves': [100, 120, 130, 140, 150, 160],
    'objective': ['binary'],
    'max_depth': [4, 5, 6, 7, 8, 9, 10],
    'learning_rate': [0.01, 0.02, 0.05, 0.1, 0.15, 0.2],
    'max_bin': [100, 120, 150, 200]
}

In [27]:
strat_kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [30]:
random_search = RandomizedSearchCV(estimator=lgb.LGBMClassifier(), param_distributions=params, n_iter=100, cv=strat_kfold, verbose=1)

In [31]:
random_search.fit(previsores, alvo)

Streaming output truncated to the last 5000 lines.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
                   estimator=LGBMClassifier(), n_iter=100,
                   param_distributions={'learning_rate': [0.01, 0.02, 0.05, 0.1,
                                                          0.15, 0.2],
                                        'max_bin': [100, 120, 150, 200],
                                        'max_depth': [4, 5, 6, 7, 8, 9, 10],
                                        'num_leaves': [100, 120, 130, 140, 150,
                                                       160],
                                        'objective': ['binary']},
                   verbose=1)

In [32]:
random_search.best_params_

{'objective': 'binary',
 'num_leaves': 120,
 'max_depth': 5,
 'max_bin': 120,
 'learning_rate': 0.05}

In [33]:
random_search.best_score_

0.8723721930243669

In [34]:
# Aplicando os melhores parâmetros
lgbm2 = lgb.LGBMClassifier(num_leaves=120,
                           max_depth=5,
                           max_bin=120,
                           learning_rate=.05,
                           objective='binary')

In [35]:
lgbm2.fit(X_tr, y_tr)

[LightGBM] [Info] Number of positive: 338, number of negative: 303
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000130 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 345
[LightGBM] [Info] Number of data points in the train set: 641, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.527301 -> initscore=0.109313
[LightGBM] [Info] Start training from score 0.109313
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

LGBMClassifier(learning_rate=0.05, max_bin=120, max_depth=5, num_leaves=120,
               objective='binary')

In [36]:
y_pred2 = lgbm2.predict(X_ts)

In [37]:
y_pred2

array([0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0])

In [38]:
print(f"Acurácia: {accuracy_score(y_ts, y_pred2)*100:.3f}")

Acurácia: 87.681


In [39]:
print(classification_report(y_ts, y_pred2))

              precision    recall  f1-score   support

           0       0.81      0.89      0.85       107
           1       0.92      0.87      0.90       169

    accuracy                           0.88       276
   macro avg       0.87      0.88      0.87       276
weighted avg       0.88      0.88      0.88       276



In [40]:
print(confusion_matrix(y_ts, y_pred2))

[[ 95  12]
 [ 22 147]]
